# Multi-Agent System for Workshop Preparation

This notebook demonstrates how to build a multi-agent system using [smolagents](https://huggingface.co/docs/smolagents), where each agent performs a specialized task. The manager agent orchestrates their collaboration to assist users in preparing for datacraft workshops based on their learning goals.

## 1. Load dependencies and environment

We start by importing required packages and loading environment variables (such as the OpenAI API key).

In [ ]:
import os
from dotenv import load_dotenv
from smolagents import CodeAgent, OpenAIServerModel

from workshop_smolagents_mcp.tools import query_events_db, web_search, summarize_url

load_dotenv(override=True)

## 2. Initialize the language model

We configure the OpenAI model that will be used across all agents.

## TODO:

Configure the OpenAI model using OpenAIServerModel

In [ ]:
model = ...

## 3. Create specialized agents

We create two agents:
- `WebAgent` can search and summarize content from the web.
- `EventsAgent` can query a local SQLite database of datacraft workshops.

## TODO:

Instanciate the web and events agents using CodeAgent

In [1]:
web_agent = ...

events_agent = ...

## 4. Create the manager agent

This agent coordinates the other two. It receives user goals and decides how to split the task between the database and web agents.

### TODO: 

Add the two agents defined above to the `managed_agents` list of the `manager_agent`.

## TODO:

Instanciate the manager agent using CodeAgent. This one has a `managed_agents` list attribute and no tools.

In [2]:
manager_agent = ...

## 5. Run an example request

We simulate a user asking to improve their skills in generative AI. The manager agent will find both workshop and web resources to match.

### TODO: 

Have fun: try to run the multi-agent system!

In [ ]:
user_goal = "Help me draft an invitation for the event which nomenclature is 2506-Geodis-GenAI4dev"
response = manager_agent.run(user_goal)
print(response)